# Punto 1

# Breast Cancer
Como se mencionó en el trabajo anterior, es preferible usar la métrica recall, puesto que minimiza el número de tumores maligos predichos, por lo que es menos probable que catalogue un tumor maligno como benigno y ponga la vida de alguien en riesgo por falta de detección.

Se tomará la base de datos preprocesada del trabajo anterior.

El código para obtener los modelos es el siguiente:

Obtención de modelos:

In [1]:
from pickle import load
from warnings import filterwarnings

filterwarnings("ignore")

with open( 'BreastCancer.pkl', 'rb' ) as f:
    X_train, X_test, y_train, y_test = load(f)

with open( 'BreastCancerModels.pkl', 'rb' ) as f:
    Models = load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'BreastCancer.pkl'

In [31]:
from pandas import DataFrame
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

results = DataFrame(columns=['kernel', 'Best C', 'Best gamma', 'cpu time', 'precision', 'recall', 'f1_score', 'auc'])

for Model in Models:
    pred = Model['model']
    params = pred.best_params_
    cpu_time = Model['time']
    
    y_pred = pred.predict(X_test)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    
    results.loc[ len( results.index ) ] = { 'kernel': Model['kernel'], 'Best C': params['classifier__C'], 
        'Best gamma': params['classifier__gamma'], 'cpu time': cpu_time, 'precision': precision, 
        'recall': recall, 'f1_score': f1, 'auc': auc }

results

,kernel,Best C,Best gamma,cpu time,precision,recall,f1_score,auc
0,linear,0.100,0.0001,52.625473,0.841584,0.934066,0.885417,0.813187
1,poly,0.001,100.0000,4926.816340,0.908046,0.868132,0.887640,0.857143
2,rbf,10000.000,0.0100,2.781268,0.921348,0.901099,0.911111,0.883242
3,sigmoid,1000.000,0.0001,2.656181,0.841584,0.934066,0.885417,0.813187


Es de considerabe que el kernel *poly* fué aquel que más tiempo consumió, teniendo en cuenta que se realizó en un servidor con 6 cores, y se hizo uso de la librería *scikit-learn-intelex*, que disminuyó considerablemente el tiempo de ejecución para cada modelo. En futuros puntos, se optará por usar *BayesSearchCV*, una extensión de *GridSearchCV* que aplica los principios de la optimización bayesiana. Cabe considerar que la mayoría del tiempo en ejecución del segundo kernel proviene de usar valores altos de gamma y C.

Por otro lado, podemos ver que el kernel rbf es un kernel ideal para la clasificación ya que posee muy buenos scores de f1 u auc, lo que indica una buena clasificación para ambas categorías. 

Código fuente para los modelos (No ejecutar):

In [ ]:
from pickle import load, dump
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from time import time

from numpy import logspace

kernels = ['linear', 'poly', 'rbf', 'sigmoid']

param_grid = {
    'classifier__C': logspace( -4, 4, 9 ),
    'classifier__gamma': logspace( -4, 4, 9 )
}

models = []

cv = StratifiedShuffleSplit( n_splits=5, test_size=0.2, random_state=37 )

for kernel in kernels:
    model = {}

    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', SVC( kernel = kernel ))
    ])

    gs = GridSearchCV( pipe, param_grid = param_grid, cv = cv, scoring = 'roc_auc', n_jobs=-1, verbose = 2 )
    start = time()
    gs.fit( X_train, y_train )
    end = time()

    model['kernel'] = kernel
    model['model'] = gs
    model['time'] = end - start
    models.append(model)

# Boston Housing

De igual manera que con el punto anterior, se usará la misma base usada.

In [32]:
with open( 'BostonHousing.pkl', 'rb' ) as f:
    X_train, X_test, y_train, y_test = load(f)

with open( 'BostonHousingModels.pkl', 'rb' ) as f:
    Models = load(f)

In [60]:
from sklearn.metrics import mean_squared_error, r2_score
from numpy import mean, sqrt, abs

results = DataFrame(columns=['kernel', 'Best C', 'Best gamma', 'Best epsilon','cpu time', 'RMSE', 'R2', 'MAPE'])

for Model in Models:
    pred = Model['model']
    params = pred.best_params_
    cpu_time = Model['time']
    
    y_pred = pred.predict(X_test)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    mape = mean(abs((y_test - y_pred) / y_test)) * 100
    
    results.loc[ len( results.index ) ] = { 'kernel': Model['kernel'], 'Best C': params['classifier__C'],
        'Best gamma': params['classifier__gamma'], 'Best epsilon': params['classifier__epsilon'], 'cpu time': cpu_time, 
        'RMSE': rmse, 'R2': r2, 'MAPE': mape }

results

,kernel,Best C,Best gamma,Best epsilon,cpu time,RMSE,R2,MAPE
0,linear,0.10,0.0001,1.000000,992.835901,6.821104,0.552228,23.027216
1,poly,0.01,10.0000,0.031623,231202.881519,8.105061,0.367792,20.460922
2,rbf,1000.00,0.1000,0.421697,47.737602,4.763599,0.781617,15.762402
3,sigmoid,10.00,0.0100,1.000000,38.299065,6.821591,0.552164,23.121607


En el caso de regresión, se comparten ciertas tendencias del caso anterior. En primer lugar, cabe aclarar que la gran diferencia de tiempo entre los dos se puede deber dada la inclusión de una variable adicional `epsilon`, lo que aumenta el espacio de búsqueda. Aún así, es notable que el kernel *poly* tomó un total de más de 60 horas, diferencia comparable con los menos de 20 minutos del kernel lineal, y menos de un minuto para los últimos dos. Así como en el caso anterior, la evaluación se retrasaba con valores de C y gamma superiores a 100. Podemos ver adicionalmente que al igual que el caso previo, el mejor kernel resultó siendo el rbf, en comparación con todos los parámetros.

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from numpy import logspace
from time import time

kernels = ['linear', 'poly', 'rbf', 'sigmoid']

param_grid = {
    'classifier__C': logspace( -4, 4, 9 ),
    'classifier__gamma': logspace( -4, 4, 9 ),
    'classifier__epsilon': logspace( -3, 0, 9 )
}

models = []

cv = ShuffleSplit( n_splits=5, test_size=0.2, random_state=37 )

for kernel in kernels:
    model = {}

    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', SVR( kernel = kernel ))
    ])

    gs = GridSearchCV( pipe, cv = cv, param_grid = param_grid, scoring = 'neg_mean_squared_error', n_jobs=-1, verbose = 2 )
    start = time()
    gs.fit( X_train, y_train )
    end = time()
    
    model['kernel'] = kernel
    model['model'] = gs
    model['time'] = end - start
    models.append(model)